# Crear tabla para respuestas de chat #
Se crea una tabla que contendra nuestros inputs y responses

In [12]:
import pixeltable as pxt
from pixeltable.functions import llama_cpp

pxt.drop_dir('llama_demo', force=True)
pxt.create_dir('llama_demo')

t = pxt.create_table('llama_demo.chat', {'input': pxt.String})

Created directory 'llama_demo'.
Created table `chat`.


Ahora agregamos a nuestra columna calculada la función de *create_chat_completition*. Vamos a usar un modelo preentrenado del repositorio Hugging Face. Para importar un modelo, llama.cpp tiene los campos de repo_id (donde se pone la url del modelo) y el repo_filename que es el nombre del repositorio. El modelo se descarga y cachea automáticamente.

En este caso se va a usar **Qwen2.5-0.5B** ya que es relativamente ligero y tiene resultados decentes. Además se usa el cuantificado Q5_K_M (5-bit). El **cuantificador Q5_K_M (5-bit)** es un método de compresión avanzado para modelos de lenguaje como los de la familia Qwen, que reduce el tamaño del modelo y los requisitos de memoria manteniendo un buen equilibrio entre precisión y rendimiento. Funciona agrupando pesos, etc. Este cuatificado trae un ecxelente balance de calidad y eficiencia.

In [13]:

# Add a computed column that uses llama.cpp for chat completion
# against the input.

messages = [
    {'role': 'system', 'content': 'You are a helpful assistant.'},
    {'role': 'user', 'content': t.input}
]

t.add_computed_column(result=llama_cpp.create_chat_completion(
    messages,
    repo_id='Qwen/Qwen2.5-0.5B-Instruct-GGUF',
    repo_filename='*q5_k_m.gguf'
))

# Extract the output content from the JSON structure returned
# by llama_cpp.

t.add_computed_column(output=t.result.choices[0].message.content)

Added 0 column values with 0 errors.
Added 0 column values with 0 errors.


UpdateStatus(num_rows=0, num_computed_values=0, num_excs=0, updated_cols=[], cols_with_excs=[])

# Test de Respuestas de Chat #

In [14]:
# Test with a simple question
t.insert([
    {'input': 'What is the capital of France?'},
    {'input': 'What are some edible species of fish?'},
    {'input': 'Who are the most prominent classical composers?'}
])

Inserting rows into `chat`: 3 rows [00:00, 1000.15 rows/s]
Inserted 3 rows with 0 errors.


UpdateStatus(num_rows=3, num_computed_values=9, num_excs=0, updated_cols=[], cols_with_excs=[])

In [15]:
t.select(t.input, t.output).collect()

input,output
What is the capital of France?,The capital of France is Paris.
What are some edible species of fish?,"There are many edible species of fish, including: 1. Tilapia (also known as cod) 2. Salmon 3. Trout 4. Pike 5. Bluefish 6. Rainbow trout 7. Flounder 8. Codfish 9. Bass 10. Tuna 11. Mahi-mahi 12. Hake 13. Codfish 14. Tilapia 15. Salmon 16. Trout 17. Pike 18. Bluefish 19. Rainbow trout 20. Flounder These fish are commonly found in many parts of the world and are popular in many cuisines around the world."
Who are the most prominent classical composers?,"The most prominent classical composers are Bach, Mozart, Beethoven, and Haydn. These composers are known for their innovative compositions, particularly in the form of the symphonies, concertos, and operas. They are also known for their use of complex harmonies and melodies, as well as their ability to convey complex ideas and emotions through their music."


# Comparación de modelos #
Vamos a probar ahora a correr un modelo un poco más pesado como lo es **Llama-3.2-1B**. Creamos la columna correspondiente para guardar su respuesta.

In [22]:
messages_teacher = [
    {'role': 'system',
     'content': 'You are a patient school teacher. '
                'Explain concepts simply and clearly.'},
    {'role': 'user', 'content': t.input}
]

t.add_computed_column(result_teacher=llama_cpp.create_chat_completion(
    messages_teacher,
    repo_id='bartowski/Llama-3.2-1B-Instruct-GGUF',
    repo_filename='*Q5_K_M.gguf'
))

t.add_computed_column(output_teacher=t.result_teacher.choices[0].message.content)

t.select(t.input, t.output_teacher).collect()

Added 3 column values with 0 errors.
Added 3 column values with 0 errors.


input,output_teacher
What is the capital of France?,"Bonjour! I'd be happy to help you with that. The capital of France is Paris. It's a beautiful city located in the north of the country, and it's known for its stunning architecture, rich history, and vibrant culture. Imagine you're in a big city, and you're looking for a place to visit. You might want to see the Eiffel Tower, the Louvre Museum, or the famous Notre-Dame Cathedral. Paris is a great place to explore, and it's a popular destination for tourists from all over the world. Does that make sense?"
What are some edible species of fish?,"As a teacher, I'd be happy to explain some edible species of fish in a simple and clear way. There are many types of fish that are safe to eat and can be a great source of protein. Here are some examples: 1. **Salmon**: Salmon is a fatty fish that's rich in omega-3 fatty acids. It's a popular choice for grilling, baking, or smoking. It's also high in protein and low in fat. 2. **Tuna**: Tuna is a meaty fish that's high in protein and low in fat. It's often canned in water or olive oil and ...... , shellfish that's high in protein and low in fat. It's often cooked in a variety of ways, including stir-fries and curries. 8. **Eel**: Eel is a type of fish that's high in protein and low in fat. It's often grilled or baked and is a popular choice for salads and sandwiches. Remember to always check the freshness and safety of the fish before consuming it. It's also a good idea to cook the fish thoroughly to avoid foodborne illness. Which of these fish would you like to learn more about?"
Who are the most prominent classical composers?,"As a teacher, I'd be happy to introduce you to some of the most prominent classical composers. Here are a few: 1. **Wolfgang Amadeus Mozart** (1756-1791): Known for his melodic and harmonic genius, Mozart is one of the most popular and influential composers of all time. He wrote over 600 works, including symphonies, operas, and chamber music. 2. **Ludwig van Beethoven** (1770-1827): A German composer who revolutionized classical music, Beethoven is famous for his symphonies, piano sonatas, ...... today. 6. **Antonio Vivaldi** (1678-1741): An Italian composer, Vivaldi is famous for his concertos, particularly his ""Four Seasons."" He's considered one of the greatest composers of the Baroque period, and his music is still widely performed today. 7. **George Frideric Handel** (1685-1759): A German-born British composer, Handel is known for his operas, oratorios, and concertos. He's considered one of the greatest composers of the Classical period, and his music is still widely performed"
